# Downloading the Data

In [141]:
import requests
from typing import List

In [142]:
dataset_uris = ["https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet",
                "https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet"]

In [143]:
filenames = []
def download_files(file_list: List) -> None:
    for url in file_list:
        filename = "data/" + url.split("/")[-1]
        filenames.append(filename)
        body = requests.get(url)
        with open(filename, 'wb') as file:
            file.write(body.content)

download_files(dataset_uris)

# Read Datasets

In [16]:
!pip install pyarrow fastparquet

  Using cached pyarrow-8.0.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.4 MB)
     |████████████████████████████████| 1.6 MB 7.8 MB/s eta 0:00:01
  Using cached fsspec-2022.3.0-py3-none-any.whl (136 kB)
     |████████████████████████████████| 1.6 MB 30.1 MB/s eta 0:00:01


In [144]:
import pandas as pd
import re

In [145]:
pd.set_option("display.precision", 2)

In [146]:
for file in filenames:
    df = pd.read_parquet(file)
    display(df.shape)
    display(df.head())
    file = re.sub(".*(tripdata)_(\d{4}-\d{2}).parquet$", r"data/\1_\2.csv", file)
    df.to_csv(file, index=0)

(1154112, 7)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


(1037692, 7)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00013,2021-02-01 00:01:00,2021-02-01 01:33:00,NaN,NaN,None,B00014
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,NaN,225.0,None,B00037


In [147]:
jan = pd.read_csv("data/tripdata_2021-01.csv", parse_dates=["pickup_datetime", "dropOff_datetime"])
feb = pd.read_csv("data/tripdata_2021-02.csv", parse_dates=["pickup_datetime", "dropOff_datetime"])

# QUESTION 1

In [148]:
print(f"There are {jan.shape[0]} records on January for this dataset")

There are 1154112 records on January for this dataset


# QUESTION 2

In [149]:
jan["duration"] = jan["dropOff_datetime"] - jan["pickup_datetime"]

In [150]:
feb["duration"] = feb["dropOff_datetime"] - feb["pickup_datetime"]

In [151]:
print(f'The mean duration for January is {jan["duration"].mean().total_seconds()/60}')

The mean duration for January is 19.167224083333334


In [152]:
jan["duration"] = jan["duration"].apply(lambda x: x.total_seconds()/60)

In [153]:
feb["duration"] = feb["duration"].apply(lambda x: x.total_seconds()/60)

# QUESTION 3

In [154]:
query_jan = (jan["duration"] > 1) & (jan["duration"] < 60)
query_feb = (feb["duration"] > 1) & (feb["duration"] < 60)

In [155]:
jan = jan[query_jan]
feb = feb[query_feb]

In [156]:
jan.shape

(1106890, 8)

In [157]:
print(f"I drop {1154112-jan.shape[0]} records")

I drop 47222 records


In [158]:
jan.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number',
       'duration'],
      dtype='object')

In [159]:
print(f'{(jan["PUlocationID"].isna().sum() / jan.shape[0])*100:.2f}% of missing values')

83.63% of missing values


In [160]:
jan["PUlocationID"] = jan["PUlocationID"].fillna("-1")
jan["DOlocationID"] = jan["DOlocationID"].fillna("-1")
feb["PUlocationID"] = feb["PUlocationID"].fillna("-1")
feb["DOlocationID"] = feb["DOlocationID"].fillna("-1")

In [161]:
print(f'{(jan[jan["PUlocationID"] == "-1"]["PUlocationID"].count() / jan.shape[0])*100:.2f}% of missing values')

83.63% of missing values


# QUESTION 4 - OHE

In [162]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [163]:
categorical = ["PUlocationID", "DOlocationID"]

In [164]:
jan[categorical] = jan[categorical].astype("str")

In [165]:
jan.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                      object
DOlocationID                      object
SR_Flag                          float64
Affiliated_base_number            object
duration                         float64
dtype: object

In [166]:
train_dicts = jan[categorical].to_dict(orient="records")

In [167]:
dv = DictVectorizer()

In [174]:
# fit_transform to train data
X_train = dv.fit_transform(train_dicts)

In [175]:
print(f"There are {len(dv.get_feature_names())} feature matrix")

There are 525 feature matrix


# QUESTION 5

In [176]:
y_train = jan["duration"]

In [177]:
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.387902642650943

# QUESTION 6

In [178]:
feb[categorical] = feb[categorical].astype("str")
val_dicts = feb[categorical].to_dict(orient='records')
# transform to validate data
X_val = dv.transform(val_dicts)

target = 'duration'
y_val = feb[target].values

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

10.897037731359287